In [ ]:
# Loading SMPLx Body Model
import torch
from os import path as osp

from human_body_prior.body_model.body_model import BodyModel
from src.configs import Configs

# Choose the device to run the body model on, cuda or cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

configs = Configs()

bm_fname = osp.join(configs.vposer_dir, "smplx_neutral_model.npz")
bm = BodyModel(bm_fname=bm_fname).to(device)

In [ ]:
# Loading VPoser VAE Body Pose Prior
from src.models import load_vposer

vp = load_vposer(configs)
vp = vp.to(device)

In [ ]:
from eval import build_evaluator

evaluator = build_evaluator(configs)
evaluator.eval()

In [ ]:
(poses_input, poses_target, embed_input, embed_target) = evaluator.dataloader.dataset[5]

embed_input = embed_input.to(device)
poses_target = poses_target.to(device)

evaluator.model.to(device)
evaluator.model.eval()

poses_pred = evaluator.generate(embed_input.unsqueeze(0)).squeeze(0)

print(poses_pred.size())
print(poses_target.size())

In [ ]:
# Get vertices and faces of a polygonal mesh model for each body pose

from human_body_prior.tools.omni_tools import copy2cpu as c2c
import trimesh

originalPoses = { "pose_body": poses_target }
recoveredPoses = { "pose_body": poses_pred }

bmodelorig = bm(**originalPoses);
bmodelreco = bm(**recoveredPoses);
vorig = c2c(bmodelorig.v)
vreco = c2c(bmodelreco.v)
faces = c2c(bm.f)

T, num_verts = vorig.shape[:-1]

In [ ]:
# Visualize one frame's body pose before (grey) and after (purple) encode-decode
fIdx = 10
verts = vorig[fIdx]
mesh1 = trimesh.base.Trimesh(verts, faces)
mesh1.visual.vertex_colors = [254, 254, 254]
verts = vreco[fIdx]
mesh2 = trimesh.base.Trimesh(verts, faces)
mesh2.visual.vertex_colors = [254, 66, 200]
mesh2.apply_translation([1, 0, 0]) # use [0, 0, 0] to overlay them on each other
meshes = [mesh1, mesh2]
trimesh.Scene(meshes).show()

In [7]:
# Visualize a temporal subsequence of poses spatially (use mouse to rotate view)
# Note that encoding followed by decoding is not a lossless process,
# It can introduce a certain amount of error all by itself

meshes = []

for fIdx in range(25):
    verts = vorig[fIdx]
    mesh1 = trimesh.base.Trimesh(verts, faces)
    mesh1.visual.vertex_colors = [254, 254, 254]
    mesh1.apply_translation([0, 0, fIdx * .07])
    meshes.append(mesh1)
    verts = vreco[fIdx]
    mesh1 = trimesh.base.Trimesh(verts, faces)
    mesh1.visual.vertex_colors = [254, 150, 200]
    mesh1.apply_translation([0, 0, fIdx * .07])
    meshes.append(mesh1)

trimesh.Scene(meshes).show()

In [8]:
# Extract and visualize 23 body joints before and after encode-decode process
# for a pose where error between original pose and decoded pose is rather large.
# Why 23 instead of 21 mentioned earlier? There are two extra joints somewhere
# that are not among the 21 rotatable body joints used by VPoser.

fIdx = 14

verts = vorig[fIdx]
mesh1 = trimesh.base.Trimesh(verts, faces)
mesh1.visual.vertex_colors = [254, 254, 254]
verts = vreco[fIdx]
mesh2 = trimesh.base.Trimesh(verts, faces)
mesh2.visual.vertex_colors = [254, 66, 200]
mesh2.apply_translation([0, 0, 0])  # use [0, 0, 0] to overlay them on each other
meshes = [mesh1, mesh2]


# Get the 23 major 3D body joints
joints = c2c(bmodelorig.Jtr[fIdx])
origjoints = joints[0:23, :]   # ignore finger joints
joints = c2c(bmodelreco.Jtr[fIdx])
recojoints = joints[0:23, :]   # ignore finger joints

print(origjoints.shape, recojoints.shape)

for i in range(origjoints.shape[0]):
    sphere = trimesh.primitives.Sphere(radius=.02, center=origjoints[i,:])
    sphere.apply_translation([1, 0, 0])
    sphere.visual.vertex_colors = [254, 254, 254]
    meshes.append(sphere)
    sphere = trimesh.primitives.Sphere(radius=.02, center=recojoints[i,:])
    sphere.apply_translation([1, 0, 0])
    sphere.visual.vertex_colors = [254, 150, 200]
    meshes.append(sphere)

trimesh.Scene(meshes).show()